md# Explingo Experiment Runner

This notebook:
1. Loads the gold-standard dataset, prepares the metrics functions, and verifies that the metric functions give the maximum score on the gold-standard dataset, and lower scores on less aligned datasets
2. Runs the prompt-design, few-shot, and bootstrap-few-shot experiments on a testing dataset

## Setup
Import necessary libraries, prepare the LLM, and load the gold-standard dataset

**Note: To run these cells, you need a `keys.yaml` file in the top-level Explingo directory with the following line:**
```yaml
openai_api_key: <your_openai_api_key>
```

In [1]:
import examples, core 
import os
import yaml
import dspy
import metrics2 as metrics
import random

In [2]:
with open(os.path.join("..", "keys.yaml"), "r") as file:
    config = yaml.safe_load(file)
    openai_api_key = config["openai_api_key"]

llm = dspy.OpenAI(model='gpt-4o', api_key=openai_api_key, max_tokens=2000)

labeled_train, labeled_eval, unlabeled_train, unlabeled_eval = examples.get_data("gold_standards.json")
train_data = labeled_train + unlabeled_train
eval_data = labeled_eval + unlabeled_eval

Some examples include gold-standard narratives; others include only a sample explanation.
- The former makes up the gold-standard dataset used for tuning the evaluation metrics and providing few-shot examples.
- The latter makes up the testing dataset used for evaluation and for bootstrapping few-shot examples

In [3]:
max_optimal_length = max([len(d.narrative) for d in labeled_train])
max_optimal_length

474

Next, we set up the evaluation metrics. We use the following metrics, all scored on a scale from 0-2:
- Accuracy: the narrative accurately describes the information in the explanation
- Fluency: the narrative is coherent and natural, as compared to the gold-standard explanations. We pass in a small list of sample narratives from the gold-standard dataset to compare against
- Conciseness: the narrative is not too long, as compared to the gold-standard explanations. For now, any narrative that is no longer than the longest gold-standard narrative will score 2
- Context Awareness: the rationalization given alongside the explanation is relevant
- Completeness: the narrative includes all relevant information from the original explanation 

TODO: Currently, we have removed the completeness metric as the accuracy metric ends up encompassing it (and the definition of complete depends on the gold-standard). We need to decide if we want to add it back in

**Note: You can set `verbose=1` to see the narratives generated, or `verbose=2` to see the explanations, narratives, and rationalizations**

In [4]:
example_good_narratives = random.sample([d.narrative for d in labeled_train], 5)
example_bad_narratives = random.sample([d.bad_narrative for d in labeled_train if hasattr(d, "bad_narrative")], 5)

exp_metrics = metrics.Metrics(
            [
                metrics.accuracy,
                metrics.fluency,
                metrics.conciseness,
                metrics.context_awareness,
            ], verbose=0, openai_key=openai_api_key,
        metric_kwargs={"conciseness": {"max_optimal_length": max_optimal_length},
                       "fluency": {"good_narratives": example_good_narratives, "bad_narratives": example_bad_narratives}}
        )

Finally, we set up the main experiment runner object

In [5]:
explingo = core.Explingo(llm=llm, context="The model predicts house prices", metric=exp_metrics,
                         labeled_train_data=labeled_train, unlabeled_train_data=unlabeled_train)

## Verifying the Evaluation Metrics

In the following code block, we verify our metric functionality by comparing average score on our gold standard dataset (used to tune the metrics) to other datasets that use different styles of explanations. We expect the gold standard average score to be very close to 2*len(metrics) (since each metric is scored on a scale of 0-2), and the other datasets to be lower.
 
TODO: We do not currently verify the context awareness metric, as the gold-standard dataset does not include a rationalization

In [6]:
metric_verification_datasets = ["gold_standards.json", "unaligned_examples_1.json", "unaligned_examples_2.json"]

# Example datasets do not include a rationalization, so we skip context awareness 
ver_metrics = metrics.Metrics(
    [
        metrics.accuracy,
        metrics.fluency,
        metrics.conciseness,
        #metrics.completeness
    ], verbose=0, openai_key=openai_api_key,
    metric_kwargs={"conciseness": {"max_optimal_length": max_optimal_length},
                   "fluency": {"good_narratives": example_good_narratives, "bad_narratives": example_bad_narratives}}
)

for dataset in metric_verification_datasets: 
    labeled_train, _, _, _ = examples.get_data(dataset, split=1)
    all_results = None
    score = 0
    for example in labeled_train:
        result = ver_metrics(example, example)
        score += result[0]
        if all_results is None:
            all_results = result[1]
        else:
            all_results += result[1]
    print(f"Dataset: {dataset}, Average score: {score/len(labeled_train)}/{len(ver_metrics.metric_funcs)*4.0}")
    print(all_results/len(labeled_train))

Dataset: gold_standards.json, Average score: 11.75/12.0
accuracy       4.00
fluency        3.75
conciseness    4.00
dtype: float64


KeyboardInterrupt: 

## Basic prompt design experiment

We begin with basic prompts. With 4 metrics (without completeness), each with a score of 0-2, the maximum score is 8. 

We generate narratives/rationalizations on `max_iters=5` sample explanations, and return the average total score.

In [ ]:
# Utility for cleaner results

def pretty_print(result):
    s = f"Total score: {result[0]}"
    s2 = ", ".join([f"{k}: {v}" for k, v in result[1].items()])
    print(f"{s} ({s2})")

In [ ]:
prompts = [
    "You are helping users understand an ML model's prediction. Given an explanation and information about the model, convert the explanation into a human-readable narrative.",
    "You are helping users who do not have experience working with ML understand an ML model's prediction. Given an explanation and information about the model, convert the explanation into a human-readable narrative. Make your answers sound as natural as possible.",
    "You are helping users understand an ML model's prediction. Given an explanation and information about the model, convert the explanation into a human-readable narrative. Be sure to explicitly mention all values from the explanation in your response.",
]

for prompt in prompts:
    print(f"Prompt: {prompt}")
    pretty_print(explingo.run_basic_prompting_experiment(eval_data, prompt=prompt, max_iters=5))
    

## Few-shot experiment

Next, we repeat the experiment with the addition of N few-shot examples from the gold-standard dataset.

In [ ]:
for i in [1, 3, 5]:
    print(f"Few-shot n: {i}")
    pretty_print(explingo.run_few_shot_experiment(eval_data, max_iters=5, n_few_shot=i))

## Bootstrapped few-shot
Next, we repeat the experiment with the addition of 3 examples bootstrapped by DSPy to optimize the evaluation metrics.

In [ ]:
for i, j in [[0, 3], [0, 5], [3, 3], [3, 5]]:
    print(f"Few-shot n: {i}, Bootstrapped n: {j}")
    pretty_print(explingo.run_bootstrap_few_shot_experiment(eval_data, max_iters=5, n_labeled_few_shot=i, n_bootstrapped_few_shot=j))